In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [10]:
# Main thread

if __name__ == '__main__':

    # Configuring paths and inputs -------------------------------------------------------------------------------------
    my_data = np.loadtxt('./../../Catalogue/binom_reg_dataset.csv', delimiter=',', dtype=str)

    my_dictionary = {}
    for i in range(len(my_data[0, :])):                                         # Converting numpy array into dictionary
         my_dictionary[my_data[0, i]] = np.array(my_data[0 + 1:, i], dtype=str)

    logit_class   = my_dictionary['LOGIT_CLASS(1-UVUP;0-UVWEAK)'].astype(int)
    redshift      = my_dictionary['Z'].astype(float)
    stellar_mass  = my_dictionary['STELLAR_MASS'].astype(float)
    
    index = np.where(redshift<=0.4)

    x1    = redshift[index]
    x2    = stellar_mass[index]
    y     = logit_class[index]              # whether this is a galaxy with uv upturn or not
    n_obs = x1.size

    regression_data = {}
    regression_data['K'] = 5      # number of betas
    regression_data['X'] = sm.add_constant(np.column_stack((x1, x1**2, x2, x2**2)))
    regression_data['N'] = n_obs
    regression_data['Y'] = y
    regression_data['LogN'] = np.log(n_obs)
    
    # Data to be plotted -------------------------------------------------------------------------------------------
    n_obs2 = 50    
    x1_sim = np.linspace(x1.min(), x1.max(), n_obs2)
    x2_sim = np.linspace(x2.min(), x2.max(), n_obs2)
    
    plot_x1, plot_x2 = np.meshgrid(x1_sim, x2_sim)  # THIS IS WHERE THE GRID IS DONE
       
    plot_x1 = plot_x1.reshape(2500, 1)
    plot_x2 = plot_x2.reshape(2500, 1)

    regression_data['X2'] = sm.add_constant(np.column_stack((plot_x1, plot_x1**2, plot_x2, plot_x2**2)))
    regression_data['N2'] = n_obs2**2
    
    print regression_data['X2'].shape
    
#     print regression_data['X'].shape
#     print regression_data['X']
#     print regression_data['X2'].shape
#     print regression_data['X2']

    # Fit: STAN code ---------------------------------------------------------------------------------------------------
    stan_code = """
    data{
        int<lower=0> N;
        int<lower=0> N2;
        int<lower=0> K;
        int Y[N];
        matrix[N,K] X;
        matrix[N2,K] X2;
//        real LogN;
    }

    parameters{
        vector[K] beta;
    }

    transformed parameters{
        vector[N] eta;
        eta = X * beta;
    }

    model{
        Y ~ bernoulli_logit(eta);
    }

    generated quantities{
        /* real LLi[N2]; */
        /* real AIC; */ 
        /* real BIC; */
        /* real LogL; */
        vector[N2] etanew;
        real<lower=0, upper=1.0> pnew[N2];
        etanew = X2 * beta;
        for (j in 1:N2){
            pnew[j] = inv_logit(etanew[j]);
            /* LLi[j] = bernoulli_lpmf(1|pnew[j]); */
        }
        /* LogL = sum(LLi); */
        /* AIC = -2 * LogL + 2 * K; */
        /* BIC = -2 * LogL + LogN * K; */
    }
#     """

    fit = pystan.stan(model_code=stan_code, data=regression_data, iter=6000, chains=4, warmup=3000, n_jobs=3)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_04e2ff1a0d956cbb6eafaa1ed143cfa0 NOW.


(2500, 5)


/home/mlldantas/anaconda3/envs/py2k7/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmplqCN5e/stanfit4anon_model_04e2ff1a0d956cbb6eafaa1ed143cfa0_6472502712130734414.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [3]:
lines = list(range(8)) + [2 * n_obs2 + 8, 2 * n_obs2 + 9, 2 * n_obs2 + 10]
output = str(fit).split('\n')

for i in lines:
    print(output[i])

Inference for Stan model: anon_model_04e2ff1a0d956cbb6eafaa1ed143cfa0.
3 chains, each with iter=5000; warmup=2000; thin=1; 
post-warmup draws per chain=3000, total post-warmup draws=9000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
beta[1]       95.47   20.25  81.49 -88.68  43.25 103.14 152.88 236.77   16.0   1.24
beta[2]       27.54    2.87  12.38   4.61  18.53  27.13  36.31  51.88   19.0   1.21
beta[3]      -58.51     7.2  29.56 -115.4 -79.18 -57.52  -36.8  -3.25   17.0   1.22
eta[99]        0.03    0.02    0.3  -0.56  -0.17   0.03   0.24   0.63  145.0   1.04
eta[100]      -0.28  5.7e-3   0.13  -0.54  -0.36  -0.28  -0.19  -0.03  498.0   1.01
eta[101]      -0.51  5.6e-3   0.12  -0.76   -0.6  -0.51  -0.43  -0.27  495.0   1.01


In [4]:
print lines

[0, 1, 2, 3, 4, 5, 6, 7, 108, 109, 110]


In [ ]:
print fit

In [5]:
output = str(pystan.misc._print_stanfit(fit, digits_summary=3)).split('\n')

In [6]:
posteriors = list(fit.extract(u'beta').items()[0])

In [7]:
betas = posteriors[1]

In [8]:
print betas.shape

(9000, 5)


In [9]:
beta0 = betas[:,0]
beta1 = betas[:,1]
beta2 = betas[:,2]
beta3 = betas[:,3]

In [ ]:
plt.subplots(1, betas.size, figsize=((5*betas.size),7), sharey=True)

for i in range(betas.size):
    plt.subplot(1,4,betas[i])
    sns.kdeplot(beta0, shade=True, c='#e6550d')
    plt.xlabel(r"$\beta_{%d}$" % i, fontsize=25)
    plt.ylabel(r"Kernel Density", fontsize=25)
    plt.tick_params('both', labelsize='20')
    
plt.tight_layout()
plt.savefig('./../Model/Results/posterios_sharey_3d_grid.pdf', dpi=100)
plt.show()

In [ ]:
plt.subplots(1,1, figsize=(25,10), sharey=True)

plot01 = plt.subplot(1,4,1)
sns.kdeplot(beta0, shade=True, c='#e6550d')
plt.xlabel(r"$\beta_{0}$", fontsize=25)
plt.ylabel(r"Kernel Density", fontsize=25)
plt.tick_params('both', labelsize='20')

plt.subplot(1,4,2)
sns.kdeplot(beta1, shade=True, c='#e6550d')
plt.xlabel(r"$\beta_{1}$", fontsize=25)
plt.tick_params('both', labelsize='20')

plt.subplot(1,4,3)
sns.kdeplot(beta2, shade=True, c='#e6550d')
plt.xlabel(r"$\beta_{2}$", fontsize=25)
plt.tick_params('both', labelsize='20')

plt.subplot(1,4,4)
sns.kdeplot(beta3, shade=True, c='#e6550d')
plt.xlabel(r"$\beta_{3}$", fontsize=25)
plt.tick_params('both', labelsize='20')

plt.tight_layout()
plt.savefig('./../Model/Results/posterios_3d_grid.pdf', dpi=100)
plt.show()

In [ ]:
output = np.array(output)

In [ ]:
new_output = output[5:-6] #removing header and footer
print new_output.shape
print new_output.size
print new_output[0].split()[0]

In [ ]:
print new_output[-1]
print new_output[-2]
print new_output[4725].split()[1][0:6]
print new_output[4725].split()[1][6:]

In [ ]:
diagnostics = []
for i in range(new_output.size):
    if len(new_output[i].split())<11:
        print i, len(new_output[i].split()),'\n'
        print new_output[i].split(), len(new_output[i].split())
        diagnostics.append(len(new_output[i].split()))
    else:
        continue
print np.unique(diagnostics)

In [ ]:
new_output[1029] = 'etanew[521]  -0.249 0.007  0.3603 -0.968  0.490 -0.253 -6.293e-5 0.4649 2627 1.0024' 
new_output[1880] = 'etanew[1372] -0.090 0.0021 0.1322  0.346 -0.179 -0.090 -1.611e-5 0.1682 3938 1.001'

In [ ]:
header_fit = output[4].split()
print header_fit

In [ ]:
header_addendum = 'parameter'
header_fit = [header_addendum] + header_fit
print header_fit

In [ ]:
new_data = header_fit
for i in range(new_output.size):
    if len(new_output[i].split())!=11: #the length of the list must be 11, in which case we connect them directly
        print "there is a problem!"
    else:
        new_output_i = np.array(new_output[i].split()).reshape(1,11)
        new_data = np.vstack((new_data, new_output_i))       

In [ ]:
import pandas as pd

In [ ]:
new_dataframe = pd.DataFrame(new_data)
new_dataframe.to_csv('./fit_results_3d_grid.csv', sep=',', header=False, index=False)

In [ ]:
print new_data.shape
print new_data[-1]

In [ ]:
betas = {}
betas['beta0'] = beta0
betas['beta1'] = beta1
betas['beta2'] = beta2
betas['beta3'] = beta3

In [ ]:
betas_dataframe = pd.DataFrame(betas)
betas_dataframe.to_csv('./Results/betas_3d_grid.csv', sep=',', header=True, index=False)

In [ ]:
pnew = list(fit.extract(u'pnew').items()[0])

In [ ]:
model_results = {}
model_results['pnew'] = pnew[1][0]
model_results['redshift'] = plot_x1.reshape(2500,)
model_results['stellar_mass'] = plot_x2.reshape(2500,)

In [ ]:
print model_results['pnew'].shape
print plot_x1.shape
print plot_x2.shape

In [ ]:
model_dataframe = pd.DataFrame(model_results)
model_dataframe.to_csv('./Results/model_prob_3d_grid.csv', sep=',', header=True, index=False)